In [87]:
with open("data_utils/input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [25]:
print(f"total characters:{len(text)}")

total characters:1115394


In [26]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [27]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [28]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

# Endocing the text into numbers
# the way we tokenize text
encode = lambda s: [stoi[c] for c in s]
# Decoding the numbers into text
decode = lambda l: ''.join([itos[i] for i in l])


print(encode("creating bigram"))
print(decode(encode("creating bigram")))

[41, 56, 43, 39, 58, 47, 52, 45, 1, 40, 47, 45, 56, 39, 51]
creating bigram


In [31]:
# Now we will start using torch here
import torch

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [33]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.dtype)
print(data[:1000])  # first 1k characters 

torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 

In [34]:
# Split the data into train and test
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [35]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [36]:
x = train_data[ :block_size]
y = train_data[1: block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} output is {target}")

when input is tensor([18]) output is 47
when input is tensor([18, 47]) output is 56
when input is tensor([18, 47, 56]) output is 57
when input is tensor([18, 47, 56, 57]) output is 58
when input is tensor([18, 47, 56, 57, 58]) output is 1
when input is tensor([18, 47, 56, 57, 58,  1]) output is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) output is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) output is 58


In [41]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 # context length

def get_batch(split):
    # batch of x and y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print(f'inputs shape {xb.shape}')
print(f'targets shape: {yb.shape}')

inputs shape torch.Size([4, 8])
targets shape: torch.Size([4, 8])


In [42]:
# Bigram Language model
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [72]:
class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # creating an loopup -logprob table to get next token -logprob
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(seld, idx, targets):

        # idx and targets are both (B, T) tensor of intergers
        logits = seld.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # Doing some shape correction which pytoch expects
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) 
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(idx, None)
            # focus only on the last time stamp
            logits = logits[:, -1, :] #becomes (B, C)
            # apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1) #(B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            # append sampled iindex to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) #(B, T+1)
        return idx



In [77]:
# checking how the model is performing without training
# and randomly generating text from it
model= BigramLM(vocab_size)
logits, loss = model.forward(xb, yb)
print(loss)
print(logits.shape)
start_idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(start_idx, 64)[0].tolist()))


tensor(4.4725, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

twtseYTkpUb!poQRViV?cW'OM
Zo,:c:fy3iC P:cVC eo--ptv.sbQHgKSy3XYF


In [79]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [81]:
batch_size = 32
max_iter = 10000
for steps in range(max_iter):

    # create a batch
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.371460199356079


In [82]:
# if we have not trained any model then we can expect
# each vocab to have -ln(1/65) = 4.17 -log probs
# we have gone from 4.47 to 2.37
# that implies our model has learnt something
# Now lets generate new

In [84]:
start_idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(start_idx, 64)[0].tolist()))



Wpuert. mars.
Morike thad te s eche Soulowin:
To e:
Br loabrew 


In [85]:
# Certainly starting looking much more like texts of the original inputs
# Thanks